Upgrading fastai to latest version

In [ ]:
!pip install fastai --upgrade

In [ ]:
from fastai.tabular.all import *
from pathlib import Path

In [ ]:
base_path = Path('../input/spaceship-titanic')

Creating training dataframe and it's basic exploration

In [ ]:
train_df = pd.read_csv(base_path/'train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.dtypes

Getting the categorical and continuous columns

In [ ]:
cat_cols = train_df.select_dtypes(include='object').columns.tolist()
cat_cols

In [ ]:
cont_cols = train_df.select_dtypes(exclude='object').columns.tolist()
cont_cols

Since *Transported* is our label, we are not going to consider in the continuous columns.

In [ ]:
cont_cols = cont_cols[:-1]
cont_cols

Splitting the training data into training and validation sets randomly

In [ ]:
splits = RandomSplitter(valid_pct=0.2, seed=20)(train_df)

In [ ]:
dls = TabularPandas(
    df = train_df,
    splits = splits,
    procs = [Categorify, FillMissing, Normalize],
    cat_names = cat_cols,
    cont_names = cont_cols,
    y_names = "Transported",
    y_block = CategoryBlock(),
).dataloaders(path='.')

In [ ]:
learn = tabular_learner(dls, metrics=accuracy, layers=[32,16,8])

In [ ]:
#learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
learn.fit(20)

In [ ]:
test_df = pd.read_csv(base_path/'test.csv')

In [ ]:
test_df.head()

In [ ]:
tst_dl = learn.dls.test_dl(test_df)

In [ ]:
preds, _ = learn.get_preds(dl=tst_dl)

In [ ]:
preds.shape

In [ ]:
preds[3,1]

In [ ]:
test_df['Transported'] = (preds[:,1] > 0.5)

In [ ]:
test_df.head()

In [ ]:
sub_df = test_df[['PassengerId', 'Transported']]

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)